# DES
- DES是使用最广泛的**对称密码**：数据加密标准（Data Encryption Standard）。<br>
- 在对称密码系统（也叫做**私钥密码系统**）中，加密和解密使用**相同的密钥**。<br>
<div>
    <img align=left src="Images/des.png"/>
</div>

## 1. DES原理
### 1.1 基本工作原理
用56位的密钥对64位长的数据块进行16轮加密处理由此得到64位长的密文。
### 1.2 基本工作流程
<div>
    <img align=left src="Images/des-process.png"/>
</div>

## 2. 流程详解
在正式讲解如何把64位明文转换为64位密文之前，我们先讲一下DES加密使用的子密钥是如何生成的。
### 2.1 子密钥生成

子密钥生成的全流程如下：
<div>
    <img align=left src="Images/des-process-key-generate.png"/>
</div>

<div>
    <img align=left src="Images/chi-key-generate-0.png"/>
</div>

#### 2.1.1 输入**64位的密钥**到子密钥算法。

<div>
    <img align=left src="Images/key-generate-1.png"/>
</div>

#### 2.1.2 略去奇偶校验位
把每个字节的第8位作为奇偶校验位,并且略去。DES的密钥由64位减至56位：

<div>
    <img align=left src="Images/key-generate-2.png"/>
</div>

#### 2.1.3  置换选择1
把56位的密钥通过置换选择1，permutation Choice one (PC-1)：

<div>
    <img align=left src="Images/key-generate-3.png"/>
</div>

#### 2.1.4 密钥一分为二
把通过置换选择1的56位密钥，分成2部分（C，D），每部分有28位

<div>
    <img align=left src="Images/key-generate-4.png"/>
</div>

#### 2.1.5 循环左移
每轮中，这两部分分别**循环左移**l位或2位。下表给出了每轮移动的位数：

<div>
    <img align=left src="Images/key-generate-5.png"/>
</div>

#### 2.1.6 置换选择2
置换选择2,permutation choice two (PC-2)，也被称为：**压缩置换**。<br>
- 置换：例如，原第14位在输出时移到了第1位。
- 压缩：第9、18、22、25以及第35、38、43、54均被略去。

<div>
    <img align=left src="Images/key-generate-6.png"/>
</div>

经过置换选择2，我们终于实现了：将56位密钥压缩成48位。

## 明文变密文
以上就是子密钥的生成。接下来，我们开始进入正题——**把64位明文转换为64位密文**。<br>

<div>
    <img align=left src="Images/des-process-showtime.png"/>
</div>

如图所示，要把64比特的明文转换成64比特的密文，需要经历：
1. 初始置换
2. 与子密钥K进行16轮迭代
3. 逆初始置换

### 2.2 初始置换
和子密钥的置换选择一样，64位的明文也要经历置换选择。

#### 2.2.1  把明文作为输入
初始置换把**64位**明文作为输入：

<div>
    <img align=left src="Images/IP1.png"/>
</div>

#### 2.2.2  初始置换明文
初始置换把明文按照原有的**64位**进行置换，最后还是**64位**：

<div>
    <img align=left src="Images/IP2.png"/>
</div>

#### 2.2.3  一分为二
把通过置换后的64位明文，分成2部分（L，R），每部分有32位

### 2.3 与子密钥K进行16轮迭代
通过2.2.3的一分为二，现在有L、R**各32位**的置换过的明文，我们拿出16轮中的一轮来看看到底这16轮迭代到底做了什么：

<div>
    <img align=left src="Images/des-encryption.jpg"/>
</div>

#### 2.3.1  扩展置换E
扩展置换E：指的是将原本的**32位块**（$R_{i-1}$）扩展到**48位**。<br>
操作：<br>
1. 把32位分成8个4位的块
2. 第 $i$ 块向左，向右各扩展一位，其中左扩展位与第 $i-1$ 块的最右一位相同，右扩展位与第 $i+1$ 块的最左一位相同

<div>
    <img align=left src="Images/EP1.png"/>
    <img align=left src="Images/E-Box.png"/>
</div>

将输入的**32位块**（$R_{i-1}$）扩展到**48位**的输出块。这样做的目的其实是实现和**子密钥K**进行**异或运算（XOR）**。(异或运算在**同位**间进行)

#### 2.3.2  与子密钥$K_i$进行异或运算（XOR）
把通过**扩展置换E**扩展的**48位块**与**子密钥K**进行**异或运算（XOR）**<br>
$a \bigoplus b$: 如果a、b两个值不相同，则异或结果为1。如果a、b两个值相同，异或结果为0。

#### 2.3.3  压缩替换S
密钥与扩展分组异或以后，将**48位**的结果进行**代替**运算。替代由**8**个代替盒**（S盒）**完成。
<br>
- 48位的输入 -> 8个6位的分组
- 每一分组对应一个S盒代替操作（共8个）
- 每一个S盒都有6位输入，4位输出

<div>
    <img align=left src="Images/S-Box.gif"/>
</div>

- 8个S盒都不同

<div>
    <img align=left src="Images/8-S-Box.gif"/>
</div>

- 每个S盒是一个4行（0到3）、16列的表（0到15）。S盒的6bit输入确定了其对应的输出在哪一行哪一列。以$S_6$盒为例：

<div>
    <img align=left src="Images/S-Box-6.png"/>
</div>

假定将S盒的**6bit**的输入标记为b1、b2、b3、b4、b5、b6，则：
- b1和b6对应行：0到3
- b2到b5对应列：0到15

##### example
假如$S_6$盒子要处理的**6bit**数据为：110011
- b1,b2,b3,b4,b,b6 = 1,1,0,0,1,1
- 负责行的b1和b6为：$11_2 = 3_{10}$
- 负责列的b2,b3,b4, b5为：$1001_2 = 9_{10}$
- 由此可见，置换的是3行9列的：$14_{10} = 1110_2$
- 110011 -> $S_6$置换 -> 1110

<div>
    <img align=left src="Images/S-Box-6-new.png"/>
</div>

- 48位块 -> S盒 -> 32位块

将**48位**压缩到**32位**。这样做的目的其实是实现和$L$**（32位块）**进行**异或运算（XOR）**。

#### 2.3.4  P盒置换

<div>
    <img align=left src="Images/P-Box.png"/>
</div>

P置换使得一个S盒的输出对下一轮多个S盒产生影响，形成**雪崩效应**：明文或密钥的一点小的变动都引起密文的较大变化。<br>
将P盒置换的结果与最初的64位分组的左半部分异或，接着开始另一轮。

#### 2.3.5  与初始置换之后的左边一半明文$L_i$进行异或运算（XOR）
$a \bigoplus b$: 如果a、b两个值不相同，则异或结果为1。如果a、b两个值相同，异或结果为0。

###   2.4 逆初始置换
再将64位的明文进行**逆初始置换**：

<div>
    <img align=left src="Images/IP3.png"/>
</div>

<div>
    <img align=left src="Images/IP-IP-1.png"/>
</div>

###   2.5 解密
- 加密和解密可使用相同的算法。
- DES解密结构与其加密结构是对称相似的，使得能用相同的函数来加密或解密每个分组。
- 二者的唯一不同之处是密钥的次序相反。

<div>
    <img align=left src="Images/IP-1-IP.png"/>
</div>